[Manual de uso do Pyomo](http://www.opl.ufc.br/files/Manual_Pyomo_2020.pdf)

[Playlist de Programação Linear - Cassius Scarpin](https://www.youtube.com/playlist?list=PLY8sfsQRSw792jf1y1KDvRYbL5QMB5CoG)

[Tópicos de matemática para cientistas de dados - Wagner Bonat](http://leg.ufpr.br/~wagner/TMCD/MN.html#programação-linear)

[Vídeo aula - Modelando o problema do mix de produção](https://www.youtube.com/watch?v=QFZ2UL2baHg&list=PLB0Bkje224EHp-IPd9i_FADBXXQcft5L6&index=10)

[Contato](https://acsjunior.com)

(Belfiore e Fávero) A empresa Venix de brinquedos está revendo seu planejamento de produção de carrinhos e triciclos. 

O lucro líquido por unidade de carrinho e triciclo produzido é de $\text{R\$}12,00$ e $\text{R\$}60,00$, respectivamente. 

As matérias primas e os insumos necessários para a fabricação de cada um dos produtos são terceirizados, cabendo à empresa os processos de usinagem, pintura e montagem. 

O processo de usinagem requer 15 minutos de mão de obra especializada por unidade de carrinho e 30 minutos por unidade de triciclo produzida. 

O processo de pintura requer 6 minutos de mão de obra especializada por unidade de carrinho e 45 minutos por unidade de triciclo produzida. 

Já o processo de montagem necessita de 6 minutos e 24 minutos para uma unidade de carrinho e de triciclo produzida, respectivamente. 

O tempo disponível por semana é de 36, 22 e 15 horas para os processos de usinagem, pintura e montagem, respectivamente. 

A empresa quer determinar quanto produzir de cada produto por semana, respeitando as limitações de recursos, de forma a maximizar o lucro líquido semanal. Formular o problema de programação linear que maximiza o lucro líquido da empresa Venix.



|  | carrinho | triciclo |
|:--- |:---:|:---:|
| <b>Lucro (R$)</b> | 12 | 60 |


|  | carrinho | triciclo | disponib. (h) |
|:--- |:---:|:---:|:---:|
| <b>Usinagem</b> | 0,25 | 0,5 | 36 |
| <b>Pintura</b> | 0,1 | 0,75 | 22 |
| <b>Montagem</b> | 0,1 | 0,4 | 15 |

### Índices / Conjuntos:

$I \colon \{1, \ldots, m\}$

$J \colon \{1, \ldots n\}$

### Parâmetros:

$c_j \colon \text{lucro líquido por unidade do produto } j \in J \text{ produzido}$

$a_{ij} \colon \text{quantidade de horas necessárias para a execução do processo } i \in I \text{ na fabricação do produto } j \in J$

$b_i \colon \text{quantidade de horas disponíveis para execução do processo } i \in I$

### Variáveis de decisão:

$x_j \colon \text{quantidade a ser fabricada do produto } j \in J \text{ por semana}$

### Modelo matemático:

$ \mathrm{max} \; z = \sum\limits_{j \in J} c_j x_{j}$

sujeito a:

$\sum\limits_{j \in J} a_{ij} x_{j} \leq b_{i}, \;\; \forall \; i \in I$

$x_{j} \geq 0 \;\; \forall j \in J$

In [1]:
import pyomo.environ as pyo
import pandas as pd

## Leitura dos dados de entrada

In [2]:
df_atividades = pd.read_csv('atividades.csv')

In [3]:
df_atividades.head()

,atividade,disponibilidade
0,usinagem,36
1,pintura,22
2,montagem,15


In [4]:
df_horas = pd.read_csv('horas_por_atividade.csv')

In [5]:
df_horas.head()

,atividade,produto,horas
0,usinagem,carrinho,0.25
1,usinagem,triciclo,0.50
2,pintura,carrinho,0.10
3,pintura,triciclo,0.75
4,montagem,carrinho,0.10


In [6]:
df_produtos = pd.read_csv('produtos.csv')

In [7]:
df_produtos.head()

,produto,lucro
0,carrinho,12
1,triciclo,60


In [8]:
lucros = df_produtos['lucro'].to_list()

In [9]:
lucros

[12, 60]

In [10]:
limites = df_atividades['disponibilidade'].to_list()

In [11]:
limites

[36, 22, 15]

In [12]:
df_horas_pivot = df_horas.pivot(index='atividade', columns='produto', values='horas').reset_index()

In [13]:
df_horas_pivot

produto,atividade,carrinho,triciclo
0,montagem,0.10,0.40
1,pintura,0.10,0.75
2,usinagem,0.25,0.50


In [14]:
df_horas_pivot = df_horas_pivot.sort_values(by='atividade', ascending=False).reset_index(drop=True)

In [15]:
df_horas_pivot

produto,atividade,carrinho,triciclo
0,usinagem,0.25,0.50
1,pintura,0.10,0.75
2,montagem,0.10,0.40


In [16]:
horas = df_horas_pivot[['carrinho', 'triciclo']].to_numpy()

In [17]:
horas

array([[0.25, 0.5 ],
       [0.1 , 0.75],
       [0.1 , 0.4 ]])

In [18]:
m = len(limites)
n = len(lucros)

In [19]:
# Declaração do modelo:

modelo = pyo.ConcreteModel()


# Índices:

modelo.I = pyo.RangeSet(m)
modelo.J = pyo.RangeSet(n)


# Parâmetros:

modelo.c = pyo.Param(modelo.J, initialize=lambda modelo, j: lucros[j-1])
modelo.a = pyo.Param(modelo.I, modelo.J, initialize=lambda modelo, i, j: horas[i-1][j-1])
modelo.b = pyo.Param(modelo.I, initialize=lambda modelo, i: limites[i-1])


# Variáveis de decisão:

modelo.x = pyo.Var(modelo.J, within=pyo.NonNegativeReals)


# Função objetivo:

def regra_z(mod):
    return pyo.summation(mod.c, mod.x)

modelo.z = pyo.Objective(rule=regra_z, sense=pyo.maximize)


# Restrições:

def regra_disp_mo(mod, i):
    return sum(mod.a[i,j] * mod.x[j] for j in mod.J) <= mod.b[i]

modelo.restr_disp_mo = pyo.Constraint(modelo.I, rule=regra_disp_mo)

In [20]:
modelo.pprint()

1 Set Declarations
    a_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    I*J :    6 : {(1, 1), (1, 2), (2, 1), (2, 2), (3, 1), (3, 2)}

2 RangeSet Declarations
    I : Dimen=1, Size=3, Bounds=(1, 3)
        Key  : Finite : Members
        None :   True :   [1:3]
    J : Dimen=1, Size=2, Bounds=(1, 2)
        Key  : Finite : Members
        None :   True :   [1:2]

3 Param Declarations
    a : Size=6, Index=a_index, Domain=Any, Default=None, Mutable=False
        Key    : Value
        (1, 1) :  0.25
        (1, 2) :   0.5
        (2, 1) :   0.1
        (2, 2) :  0.75
        (3, 1) :   0.1
        (3, 2) :   0.4
    b : Size=3, Index=I, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :    36
          2 :    22
          3 :    15
    c : Size=2, Index=J, Domain=Any, Default=None, Mutable=False
        Key : Value
          1 :    12
          2 :    60

1 Var Declarations
    x : Size=2, Index=J


In [21]:
# resultado = pyo.SolverFactory('glpk').solve(modelo)
resultado = pyo.SolverFactory('gurobi').solve(modelo)

In [22]:
resultado.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x3
  Lower bound: 2040.0
  Upper bound: 2040.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of binary variables: 0
  Number of integer variables: 0
  Number of continuous variables: 3
  Number of nonzeros: 7
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Return code: 0
  Message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject 

In [23]:
modelo.x.pprint()

x : Size=2, Index=J
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :  70.0 :  None : False : False : NonNegativeReals
      2 :     0 :  20.0 :  None : False : False : NonNegativeReals


In [24]:
modelo.x[1]()

70.0

In [25]:
modelo.x[2]()

20.0

In [26]:
modelo.z()

2040.0